In [99]:
from openhunt.logparser import winlogbeat
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [100]:
win = winlogbeat()
spark = SparkSession.builder.appName("Process").config("spark.sql.caseSensitive", "True").getOrCreate()

In [101]:
process_injection = win.extract_nested_fields("dll_injection_2020-01-12160439.json",spark)

[+] Processing a Spark DataFrame..
[+] Reading Mordor file..
[+] Processing Data from Winlogbeat version 6..
[+] DataFrame Returned !


In [102]:
process_injection.createOrReplaceTempView("process_injection")

In [105]:
DLL_Injection = spark.sql(
'''
SELECT 
    b.computer_name,
    b.SourceImage,
    ltrim('0x', b.GrantedAccess) as granted_access,
    b.TargetImage,
    b.SourceProcessId,
    b.SourceProcessGUID,
    c.NewThreadId,
    b.TargetProcessId,
    b.TargetImage,
    c.StartFunction
FROM process_injection c
JOIN process_injection b
    ON c.SourceProcessGuid = b.SourceProcessGUID
    AND b.channel = "Microsoft-Windows-Sysmon/Operational"
    AND b.event_id = 10
WHERE c.channel = "Microsoft-Windows-Sysmon/Operational"
    AND c.event_id = 8
    AND c.StartFunction = "LoadLibraryA" OR c.StartFunction = "LoadLibraryW"
'''
)


In [106]:
df = DLL_Injection.select("computer_name", conv(DLL_Injection.granted_access, 16, 10).alias("granted_access"), "SourceImage", "SourceProcessGUID", "NewThreadId", "TargetProcessId", "TargetImage")

In [107]:
df.createOrReplaceTempView("df")

In [108]:
eid_7_8_10_minimal_privileges = spark.sql(
'''
SELECT * FROM df WHERE ((INT(granted_access) & 40) == 40) -- 40 is decimal for 0x0028. The minimal privileges you need to access process handle
'''
).show(1,False)

+---------------+--------------+---------------------------------------------------------+--------------------------------------+-----------+---------------+-------------------------------+
|computer_name  |granted_access|SourceImage                                              |SourceProcessGUID                     |NewThreadId|TargetProcessId|TargetImage                    |
+---------------+--------------+---------------------------------------------------------+--------------------------------------+-----------+---------------+-------------------------------+
|HR001.shire.com|2047999       |C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|{03ba39f5-50b2-5ce0-0000-00109995c501}|7940       |3124           |C:\Windows\system32\notepad.exe|
+---------------+--------------+---------------------------------------------------------+--------------------------------------+-----------+---------------+-------------------------------+

